## Introduction
Java supports two types of streams (is an ordered sequence of bytes of indeterminate length):
- **byte streams** : read data in bytes
- **character streams** : read data as characters

Separate classes are there for reading data and writing data. For byte streams, `InputStream` and `OutputStream` abstract classes are at the top of the hierarchy. For character stream `Reader` and `Writer` are at the top of the hierarchy.

All the above mentioned stream abstract classes implement `Closeable` and must be explicitly closed when we are done using the stream. Use of *try-with-resources* block would close the stream automatically (`Closeable` extends `AutoCloseable` whose `close()` method is called automatically when exiting a *try-with-resources* block).

There can be multiple different sources for streams - file, network, arrays, etc. Though `Reader` and `Writer` are more geared towards string based source.

## Byte Stream
`InputStream` defines the following interface to read contents from stream:

In [ ]:
public class InputStream implements Closeable {
    // Reads next byte of data from stream, returns -1 if end of stream is reached
    // Value of returned number ranges from 0-255
    public abstract int read() throws IOException;
    
    // Reads b.length number of bytes from the stream into the b array. Returns the
    // number of bytes read. Works by repeatedly calling the read method.
    public int read(byte[] b) throws IOException { 
        // Impl 
    }
    
    // Reads len number of bytes from the stream into the b array. The array b is 
    // filled starting from an offset off
    public int read(byte[] b, int off, int len) throws IOException {
        // Impl
    }
    
    // Skips over specified number of bytes, returning the actual number of bytes skipped
    public long skip(long n) throws IOException {
        // Impl
    }
    
    // Closes the stream
    public void close() throws IOException {}
    
    // Other methods
}

`InputStream` has the following hierarchy:
```
InputStream
        |
        +--- FileInputStream: obtain bytes from a file
        +--- AudioInputStream: obtain bytes from an audio stream
        +--- ObjectInputStream: for reading serialized objects
        +--- FilterInputStream: transforms other input streams
        |                   |
        |                   +--- BufferedInputStream
        |                   +--- ZipInputStream: reading compressed and uncompressed ZIP file
        +--- PipedInputStream
        +--- ...                      
```

`OutputStream` similarly is defined as:

In [ ]:
public abstract class OutputStream implements Closeable, Flushable {
    // Writes the specified byte to this output stream. The eight low-order bits
    // of the input b is written
    public abstract void write(int b) throws IOException;
    
    // Writes b.length number of bytes from b to the output stream.
    public void write(byte[] b) throws IOException {
        // Impl
    }
    
    // Writes len number of bytes from b to the output stream starting from
    // the offset index in the byte array
    public void write(byte[] b, int off, int len) throws IOException {
        // Impl
    }
    
    // Flushes this output stream and forces any buffered output bytes to be written out. 
    public void flush() throws IOException {}
    
    // Closes the output stream
    public void close() throws IOException {}
}

Hierarchy of an `OutputStream` is:
```
OutputStream
        |
        +--- FileOutputStream
        +--- ObjectOutputStream
        +--- FilterOutputStream
        |                   |
        |                   +--- BufferedOutputStream
        |                   +--- ZipOutputStream
        +--- PipedOutputStream
        +--- ...                      
```

### Some Use Cases
1. Reading text file using `InputStream`. Reading a utf-8 encoded file with foreign characters will not give correct result if characters are outside ASCII range.  

In [ ]:
public void readTextFile(String path) {
    try(InputStream in = new FileInputStream(new File(path))){
        int data = -1;
        while((data = in.read())!=-1) {
            System.out.print((char)data);    // Even though char is two bytes big
        }                                    // data will always be 1 byte long
    } catch (FileNotFoundException e) {
        e.printStackTrace();
    } catch (IOException e) {
        e.printStackTrace();
    }
}

/*
Data is read as int and is in range 0 to 255 (if EOF has not been reached).
It can be saved as a byte but that would cause overflow. To convert back to int
use the following formula: int i = (b >= 0) ? b : 256 + b;

Example:
int i = 255;
byte b = (byte) b;
int i_clone = 256 + b; // = 255
*/

2. Reading header of a wav file. A wave file's header is always "RIFF".  

In [ ]:
public void readWaveHeader(String path) {
    try(InputStream in = new FileInputStream(new File(path))){
        byte[] headerBytes = new byte[4];
        in.read(headerBytes, 0, 4);
        for(int i=0;i<4;i++) {
            System.out.print((char)headerBytes[i]);
        }
    } catch (FileNotFoundException e) {
        e.printStackTrace();
    } catch (IOException e) {
        e.printStackTrace();
    }
}

3. Writing to a file  

In [ ]:
public static void writeTables(String path, int start, int end) {
    try(OutputStream out = new FileOutputStream(new File(path))){
        for(int i=start;i<=end;i++) {
            for(int j=1;j<10;j++) {
                int result = i*j;
                String line = "" + i + " X " + j + " = " + result + System.lineSeparator();
                out.write(line.getBytes());  // getBytes() uses platform's default encoding
            }
        }
    } catch (FileNotFoundException e) {
        e.printStackTrace();
    } catch (IOException e) {
        e.printStackTrace();
    }
}

It is often faster to write data in large chunks than to write it byte by byte. Therefore, `write(byte[] data` version of `write` is preferred. Performance issue can also be caused if we write too much data. It is better to write data in small chunks - 1024, 2048 or 4096 bytes for file IO and 128 or 256 bytes for network IO.

4. Sample implementation of `InputStream`

In [ ]:
public class RandomInputStream extends InputStream {
    private Random random = new Random();
    private boolean closed;

    @Override
    public int read() throws IOException {
        checkOpen();
        int temp = random.nextInt() % 256;
        return Math.abs(temp);
    }

    @Override
    public int read(byte[] data, int offset, int length) throws IOException {
        checkOpen();
        byte[] temp = new byte[length];
        random.nextBytes(temp);
        System.arraycopy(temp, 0, data, offset, length);
        return length;
    }

    @Override
    public int read(byte[] data) throws IOException {
        checkOpen();
        random.nextBytes(data);
        return data.length;
    }
    
    public void close() {
        closed = true;
    }

    private void checkOpen() throws IOException {
        if (closed) {
            throw new IOException("Stream closed");
        }
    }
}

## Buffered Byte Stream
If multiple reads or writes are to be made to a stream, it is better to use Buffered Input/Output stream. Buffered streams maintain a small byte buffer (8192 bytes for `BufferedInputStream`) to/from which bytes are written/read. Internally, it calls `read(byte[] b, int off, int len)`. We supply a stream as constructor parameter to buffered stream.

The example below buffers data from a file source:

In [ ]:
public static void readTextFile(String path) {
    try(InputStream in = new BufferedInputStream(new FileInputStream(new File(path)))){
        int data = -1;
        while((data = in.read()) != -1) {
            System.out.print((char)data);
        }
    } catch (FileNotFoundException e) {
        e.printStackTrace();
    } catch (IOException e) {
        e.printStackTrace();
    }
}

In case of `FileInputStream` even though using a `BufferedInputStream` is the best approach it is probably not the fastest. To achieve a faster read we can maintain our own custom buffer:

In [ ]:
public static void readTextFile(String path) {
    try(InputStream in = new FileInputStream(args[0])) {
        byte buf[] = new byte[2048];
        int n = 0;
        
        // the while loop fills in the buffer
        while((n = in.read(buf)) != -1) {
            // the for loop consumes the filled buffer
            for(int i=0; i<n; i++) {
                System.out.println((char) buf[i]);
            }
        }
    } catch (FileNotFoundException e) {
        e.printStackTrace();
    } catch (IOException e) {
        e.printStackTrace();
    }
}

One additional benefit of using `BufferedInputStream` is that these classes support `mark` and `reset` operations.

## Character Streams
`Reader` defines the following interface:

In [ ]:
public abstract class Reader implements Readable, Closeable {
    // Reads a character from the input stream. The character is read as an integer
    // in the range 0 to 65535
    public int read() throws IOException {
        // Impl
    }
    
    // Below two methods reads characters into the specified char array
    public int read(char[] cbuf) throws IOException {
        // Impl
    }
    public abstract int read(char[] cbuf, int off, int len);
    
    // Closes the underlying stream
    public abstract void close() throws IOException;
    
    // Other methods like mark and reset
}

`Reader` hierarchy
```
Reader
    |
    +--- BufferedReader
    +--- InputStreamReader: bridge between byte and character stream. Lets you set encoding.
    |                   |
    |                   +--- FileReader
    +--- StringReader
    +--- FilterReader
    +--- ...
```
`Writer` is defined as following:

In [ ]:
public abstract class Writer implements Appendable, Closeable, Flushable {
    // Writes lower 16 bits of the given integer value
    public void write(int c) throws IOException {
        // Impl
    }
    
    // Writes an array of characters
    public void write(char[] cbuf) throws IOException {
        // Impl
    }
    public abstract void write(char[] cbuf, int off, int len) throws IOException;
    
    // Writes a string value
    public void write(String str) throws IOException {
        // Impl
    }
    
    public abstract void flush() throws IOException;

    public abstract void close() throws IOException;

    // Other methods
}

`Writer` hierarcy:
```
Writer
    |
    +--- BufferedWriter
    +--- OutputStreamWriter
    |                   |
    |                   +--- FileWriter
    +--- StringWriter
    +--- FilterWriter
    +--- ...
```

### Use Cases
1. Reading a text file. `FileReader` uses platform default encoding, which isn't always a great idea. It may fail to read foreign characters properly.

In [ ]:
public static void readSomeCharacters(String path, int amount) {
    try(Reader r = new FileReader(path)){
        char[] data = new char[amount];
        r.read(data,0,amount); // Uses platform's default encoding
        System.out.print(data);
    } catch (FileNotFoundException e) {
        e.printStackTrace();
    } catch (IOException e) {
        e.printStackTrace();
    }
}

2. `BufferedReader` is the better choice in most cases. It can read entire line at a time. Above example wrapped in `BufferedReader`:

In [ ]:
public static void readLines(String path) {
    try(BufferedReader br = new BufferedReader(new FileReader(path))){
        String line = null;
        while((line=br.readLine()) != null) {
            System.out.println(line);
        }
    } catch (FileNotFoundException e) {
        e.printStackTrace();
    } catch (IOException e) {
        e.printStackTrace();
    }
}

3. Writing UTF-16 encoded content.

In [ ]:
public void writeLines(String path) {
    try(BufferedWriter bw = new BufferedWriter(
            new OutputStreamWriter(
                    new FileOutputStream(
                            new File(path)), "UTF-16"))){
        String text = "تشكيل الحروف";
        bw.write(text);
    } catch (UnsupportedEncodingException e) {
        e.printStackTrace();
    } catch (FileNotFoundException e) {
        e.printStackTrace();
    } catch (IOException e) {
        e.printStackTrace();
    }
}

## Non-Blocking IO

## File System

Java NIO package introduced new classes and ways to interact with files. The `Path` replaces `File` and `Paths` and `Files` classes introduce new methods to to perform file operations.

<img src="images/files_and_paths.png" width=600 height=auto />

**`Path`:** represents a file or a directory.  
**`Paths`:** contains a static method to create `Path` instance  
**`Files`:** contains static methods to operate on `Path` instances

In [ ]:
// Create a file Path. Equivalent to calling:
// Paths.get("D:", "Codes", "Eclipse Workspace", "Testbed", "pom.xml")
Path json = Paths.get("D:\\file.json");

// Create a directory Path
Path temp = Paths.get("C:\\temp");

// Read and Write from the file
if (!Files.exists(json)) {
    Files.createFile(json);
    
    // Write string to a file
    Files.writeString(json, """
            {
                "status": 200,
                "message": "Success!"
            }
            """);
    
    // Read all lines of the file as List
    System.out.println(Files.readAllLines(json));
}

| **Method**                                                                       | **Description**                                                                                       |
|----------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------|
| `copy(Path source, Path target)`                                                 | Copies a file from the source path to the target path.                                                |
| `delete(Path path)`                                                              | Deletes the file or directory at the specified path.                                                  |
| `exists(Path path, LinkOption... options)`                                       | Tests whether a file or directory exists at the specified path.                                       |
| `isDirectory(Path path, LinkOption... options)`                                  | Tests whether the specified path is a directory.                                                      |
| `isRegularFile(Path path, LinkOption... options)`                                | Tests whether the specified path is a regular file.                                                   |
| `move(Path source, Path target)`                                                 | Moves a file from the source path to the target path.                                                 |
| `newDirectoryStream(Path dir)`                                                   | Returns a directory stream to iterate over files in the specified directory.                          |
| `readAllLines(Path path, Charset cs)`                                            | Reads all lines from a file at the given path into a list of strings, using the specified charset.    |
| `write(Path path, Iterable<? extends CharSequence> lines, Charset cs)`           | Writes lines of text to the specified file at the given path.                                         |
| `createDirectories(Path dir, FileAttribute<?>... attrs)`                         | Creates a directory at the given path, including any necessary but nonexistent parent directories.    |
| `createFile(Path path, FileAttribute<?>... attrs)`                               | Creates a new empty file at the specified path.                                                       |
| `isHidden(Path path)`                                                            | Tests whether the file at the given path is hidden.                                                   |
| `size(Path path)`                                                                | Returns the size of the file in bytes at the given path.                                              |
| `walk(Path start, FileVisitOption... options)`                                   | Walks the file tree starting at the given path and applies a file visitor.                            |
| `setLastModifiedTime(Path path, FileTime time)`                                  | Sets the last modified time of the file at the specified path.                                        |
| `readAttributes(Path path, Class<T> type, LinkOption... options)`                | Reads the attributes of a file at the given path.                                                     |
| `lines(Path path, Charset cs)`                                                   | Returns a stream of lines read from the file at the specified path.                                   |

Iterating over contents of a directory can be achieved using the `walk` method:

In [ ]:
// Reading directories
if (Files.exists(temp)) {
    Files.walkFileTree(temp, new FileVisitor<Path>() {
        @Override
        public FileVisitResult preVisitDirectory(Path dir, BasicFileAttributes attrs) throws IOException {
            System.out.println("Going to visit directory:" + dir);
            return FileVisitResult.CONTINUE;
        }

        @Override
        public FileVisitResult visitFile(Path file, BasicFileAttributes attrs) throws IOException {
            System.out.println("Reading contents of file:" + file + " contents:" +
                    String.join("", Files.readAllLines(file)));
            return FileVisitResult.CONTINUE;
        }

        @Override
        public FileVisitResult visitFileFailed(Path file, IOException exc) throws IOException {
            System.out.println("Unable to access file:" + file);
            return FileVisitResult.CONTINUE;
        }

        @Override
        public FileVisitResult postVisitDirectory(Path dir, IOException exc) throws IOException {
            System.out.println("Visited directory:" + dir);
            return FileVisitResult.CONTINUE;
        }
    });
}